 ## **Importando e instalando libs necessárias**

In [ ]:
!pip install -qU langchain-groq
!pip install -U langchain-community

In [36]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain-openai


In [54]:
from google.colab import userdata
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import getpass
import os

## **Criando meu webscrapper com o embedder da Hugging face**

In [55]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

def url_to_vector(url):
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

  loader = WebBaseLoader(url)
  docs = loader.load()

  text_splitter = RecursiveCharacterTextSplitter()
  documents = text_splitter.split_documents(docs)
  vector = FAISS.from_documents(documents, embeddings)
  retriever = vector.as_retriever()
  return retriever

In [59]:

os.environ["GROQ_API_KEY"] = userdata.get('Croq')

from langchain_groq import ChatGroq

llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")

In [60]:
#Testando meu prompt sem apoio dos dados do RAG

def oscar(filme, ano, llm):
  prompt = PromptTemplate(
      input_variables= ['filme', 'ano'], #os dois parâmetros que ele vai aceitar são o filme e o ano
      template = "Quantos oscars o filme {filme} ganhou em {ano}"
  )

  oscar_chain = LLMChain(llm=llm, prompt= prompt)

  response = oscar_chain({'filme': filme, 'ano': ano})

  return response

In [66]:
if __name__=="__main__":
  response = oscar('Oppenheimer', 2024, llm) # O diferencial do vídeo para o meu é que eu uso um modelo mais atualizado
  print(response['text'])

O filme "Oppenheimer", dirigido por Christopher Nolan, ganhou 7 Oscars em 2024. Os prêmios foram:

- Melhor Filme 
- Melhor Diretor (Christopher Nolan)
- Melhor Ator (Cillian Murphy)
- Melhor Ator Coadjuvante (Robert Downey Jr.)
- Melhor Fotografia
- Melhor Edição de Som
- Melhor Mixagem de Som


## **Testando meu modelo agora com o apoio do RAG com o site da wikipedia**

In [65]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente útil."),
    ("human", "Contexto:\n{context}\n\nPergunta: {input}") #Meu prompt que aceita um contexto e o input
])

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = url_to_vector("https://pt.wikipedia.org/wiki/Oppenheimer_(filme)")
retriever_chain = create_retrieval_chain(retriever, document_chain)
response = retriever_chain.invoke({"input":"Quantos oscars o filme Oppenheimer ganhou em 2024"})
print(response['answer'])

O filme "Oppenheimer" ganhou 7 prêmios no Oscar 2024, incluindo Melhor Filme, Melhor Diretor, Melhor Ator, Ator Coadjuvante, Melhor Fotografia, Melhor Trilha Sonora Original e Melhor Edição.
